In [1]:
import pygame

from pygame.locals import *

pygame.init()

import random

pygame 2.5.2 (SDL 2.28.3, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
screen_width = 400
screen_height = 400

screen = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption('2-0-4-8')

line_width = 8
clicked = False
pos = []
player = 1

green = (0, 255, 0)
red = (255, 0, 0)
blue = (0, 0, 255)
gray = (200, 200, 200)

game_over = False

font = pygame.font.SysFont(None, 40)

again_rect = Rect(screen_width // 2 - 80, screen_height // 2, 160, 50)

In [3]:
def draw_grid():
    bg = (255, 255, 225)
    grid = (50, 50, 50)
    screen.fill(bg)
    for x in range(1,4):
        pygame.draw.line(screen, grid, (0, x *100),
                         (screen_width, x * 100), line_width)
        pygame.draw.line(screen, grid, (x * 100, 0),
                         (x * 100, screen_height), line_width)

In [4]:
values = [2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096]

field = []

for x in range(4):
    row = [0]*4
    field.append(row)

print(field)


def upgrade(value):
    pass

field[1][1] = 4
field[0][3] = 16
field[1][3] = 8
field[0][0] = 16

[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]


In [5]:
def draw_value(winner):
    win_text = 'Player ' + str(winner) + ' wins!'
    win_img = font.render(win_text, True, blue)
    pygame.draw.rect(screen, green,  (screen_width // 2 - 100, screen_height // 2 - 60, 200, 50))
    screen.blit(win_img, (screen_width // 2 - 100, screen_height // 2 - 50))

In [6]:
def draw_markers():
    x_pos = 0
    for x in field:
        y_pos = 0
        for y in x:
            if y == 0:
                pass
            else:
                win_img = font.render(str(y), True, blue)
                # pygame.draw.rect(screen, green,  (x_pos*100 + 50, y_pos*100 + 50, 200, 50))
                screen.blit(win_img, (x_pos*100 + 50, y_pos*100 + 50))
            y_pos += 1
        x_pos += 1
        
rand_gen = False
def place_new():
    global rand_gen
    while rand_gen:
        x = random.randint(0,3)
        y = random.randint(0,3)
        if field[x][y] == 0:
            field[x][y] = 2
            print('2 on ', x+1, 'x', y+1)
            rand_gen = False

In [7]:

run = True

counter_reverse = list(reversed(list(range(4))))
counter = list(range(4))

while run:
    # pygame.time.delay(100)
    draw_grid()
    draw_markers()
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            run = False
            
        elif event.type == pygame.KEYUP:
            if event.key == pygame.K_s:
                rand_gen = True
                x_pos = 0
                for column in counter:
                    y_pos = 0
                    for row in counter:
                        if field[x_pos][y_pos] != 0:
                            if y_pos < 3:
                                if field[x_pos][y_pos+1] == 0:
                                    field[x_pos][y_pos+1] = field[x_pos][y_pos]
                                    field[x_pos][y_pos] = 0
                                elif field[x_pos][y_pos+1] == field[x_pos][y_pos] and field[x_pos][y_pos] !=0:
                                    field[x_pos][y_pos+1] = 2*field[x_pos][y_pos]
                                    field[x_pos][y_pos] = 0
                        y_pos += 1
                    x_pos += 1
                
                for i in counter:
                    x_pos = 0
                    for column in counter:
                        y_pos = 0
                        for row in counter:
                            if field[x_pos][y_pos] != 0:
                                if y_pos < 3:
                                    if field[x_pos][y_pos+1] == 0:
                                        field[x_pos][y_pos+1] = field[x_pos][y_pos]
                                        field[x_pos][y_pos] = 0
                            y_pos += 1
                        x_pos += 1
                place_new()
            
            if event.key == pygame.K_w:    
                print('key w')
                rand_gen = True
                x_pos = 3
                for column in counter_reverse:
                    y_pos = 3
                    
                    for row in counter_reverse:
                        if field[x_pos][y_pos] != 0:
                            if y_pos > 0:
                                if field[x_pos][y_pos-1] == 0:
                                    field[x_pos][y_pos-1] = field[x_pos][y_pos]
                                    field[x_pos][y_pos] = 0
                                elif field[x_pos][y_pos-1] == field[x_pos][y_pos] and field[x_pos][y_pos] !=0:
                                    field[x_pos][y_pos-1] = 2*field[x_pos][y_pos]
                                    field[x_pos][y_pos] = 0
                        y_pos -= 1
                    x_pos -= 1
                for i in counter:
                    x_pos = 3
                    for column in counter_reverse:
                        y_pos = 3
                        
                        for row in counter_reverse:
                            if field[x_pos][y_pos] != 0:
                                if y_pos > 0:
                                    if field[x_pos][y_pos-1] == 0:
                                        field[x_pos][y_pos-1] = field[x_pos][y_pos]
                                        field[x_pos][y_pos] = 0
                            y_pos -= 1
                        x_pos -= 1
                place_new()
            
            if event.key == pygame.K_d:   
                rand_gen = True
                x_pos = 0
                for column in counter:
                    y_pos = 0
                    for row in counter:
                        if field[x_pos][y_pos] != 0:
                            if x_pos < 3:
                                if field[x_pos+1][y_pos] == 0:
                                    field[x_pos+1][y_pos] = field[x_pos][y_pos]
                                    field[x_pos][y_pos] = 0
                                elif field[x_pos+1][y_pos] == field[x_pos][y_pos] and field[x_pos][y_pos] !=0:
                                    field[x_pos+1][y_pos] = 2*field[x_pos][y_pos]
                                    field[x_pos][y_pos] = 0
                        y_pos += 1
                    x_pos += 1
                
                for i in counter:
                    x_pos = 0
                    for column in counter:
                        y_pos = 0
                        for row in counter:
                            if field[x_pos][y_pos] != 0:
                                if x_pos < 3:
                                    if field[x_pos+1][y_pos] == 0:
                                        field[x_pos+1][y_pos] = field[x_pos][y_pos]
                                        field[x_pos][y_pos] = 0
                            y_pos += 1
                        x_pos += 1
                place_new()
            
            if event.key == pygame.K_a:   
                rand_gen = True
                x_pos = 3
                for column in counter_reverse:
                    y_pos = 3
                    
                    for row in counter_reverse:
                        print(column, row, field[column][row])
                        if field[x_pos][y_pos] != 0:
                            if x_pos > 0:
                                if field[x_pos-1][y_pos] == 0:
                                    field[x_pos-1][y_pos] = field[x_pos][y_pos]
                                    field[x_pos][y_pos] = 0
                                elif field[x_pos-1][y_pos] == field[x_pos][y_pos] and field[x_pos][y_pos] !=0:
                                    field[x_pos-1][y_pos] = 2*field[x_pos][y_pos]
                                    field[x_pos][y_pos] = 0
                        y_pos -= 1
                    x_pos -= 1
                for i in counter: 
                    x_pos = 3   
                    for column in counter_reverse:
                        y_pos = 3
                        for row in counter_reverse:
                            if field[x_pos][y_pos] != 0:
                                if x_pos > 0:
                                    if field[x_pos-1][y_pos] == 0:
                                        field[x_pos-1][y_pos] = field[x_pos][y_pos]
                                        field[x_pos][y_pos] = 0
                            y_pos -= 1
                        x_pos -= 1    
                place_new()
            
            
    pygame.display.update()

pygame.quit()

key w
2 on  1 x 3
2 on  2 x 4
2 on  3 x 3
key w
2 on  2 x 2
3 3 0
3 2 4
3 1 8
3 0 4
2 3 0
2 2 4
2 1 2
2 0 32
1 3 0
1 2 4
1 1 4
1 0 32
0 3 0
0 2 4
0 1 4
0 0 32
2 on  1 x 4
key w
2 on  3 x 2
3 3 0
3 2 0
3 1 0
3 0 0
2 3 0
2 2 0
2 1 2
2 0 0
1 3 0
1 2 0
1 1 8
1 0 4
0 3 0
0 2 2
0 1 16
0 0 32
2 on  4 x 2
3 3 0
3 2 0
3 1 2
3 0 0
2 3 0
2 2 0
2 1 2
2 0 0
1 3 0
1 2 0
1 1 4
1 0 4
0 3 0
0 2 2
0 1 16
0 0 32
2 on  1 x 4
2 on  3 x 1
key w
2 on  4 x 2
key w
2 on  2 x 2
3 3 0
3 2 0
3 1 0
3 0 2
2 3 0
2 2 0
2 1 0
2 0 4
1 3 0
1 2 0
1 1 2
1 0 8
0 3 0
0 2 4
0 1 16
0 0 32
2 on  2 x 4
key w
2 on  2 x 3
2 on  1 x 4
key w
2 on  2 x 2
2 on  3 x 4
key w
2 on  1 x 3
3 3 0
3 2 4
3 1 2
3 0 16
2 3 0
2 2 2
2 1 16
2 0 32
1 3 0
1 2 2
1 1 16
1 0 4
0 3 0
0 2 4
0 1 16
0 0 4
2 on  4 x 4
2 on  3 x 3
3 3 2
3 2 8
3 1 2
3 0 16
2 3 2
2 2 2
2 1 16
2 0 32
1 3 2
1 2 2
1 1 16
1 0 4
0 3 2
0 2 2
0 1 16
0 0 4
2 on  2 x 4
key w
2 on  4 x 4
2 on  2 x 4
3 3 4
3 2 8
3 1 2
3 0 16
2 3 4
2 2 4
2 1 16
2 0 32
1 3 2
1 2 4
1 1 16
1 0 4
0 3 2
0 2 4

In [8]:
print(range(4))

for x in range(4):
    print(x)
    
type(list(range(4)))

print(list(reversed(list(range(4)))))

range(0, 4)
0
1
2
3
[3, 2, 1, 0]
